In [1]:
# # !pip install --upgrade langchain
# # !pip install --upgrade langchain-groq
# # !pip install -U langchain-community
# # !pip install python-docx pypandoc
# # !pip install python-docx docx2pdf

# # !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
# !pip install cryptography

In [2]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFDirectoryLoader, PyPDFLoader, WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain.agents.agent_toolkits import ZapierToolkit
from docx import Document
import pypandoc
from langchain.utilities.zapier import ZapierNLAWrapper
from langchain.agents import initialize_agent , create_structured_chat_agent, AgentType
import random
from groq import Groq
from QuoteUtils import CtxMgr

ctxmgr = CtxMgr()

api_keys = ['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','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',
            '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','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']

client = Groq(
    
    api_key = random.choice(api_keys)
)

llm = ChatGroq(groq_api_key = client.api_key,
               model_name = "llama3-70b-8192")

In [3]:
from cryptography.fernet import Fernet
import base64
from dotenv import load_dotenv
import os

load_dotenv()
with open("D:\\DEV\\WebdevFolder\\key.txt", 'r') as file:
    k = file.read().strip()[2:-1]
    

encoded_encrypted_api_key = os.getenv("ENCRYPTED_API_KEY")
encrypted_api_key = base64.b64decode(encoded_encrypted_api_key)
decipher_suite = Fernet(k)


In [4]:
from QuoteUtils import initialize_web_search_agent, initialize_quote_bot
llm.temperature = 0.7
cc_out = initialize_quote_bot(client,llm)

Thank you for providing the address of the property, which is 3210 Greenstone Way, Herndon, VA 20171. I also understand that it's a residential property and it's approximately 40 years old, built in 1980.

Let's move on to the next question. What is the square footage of the area that you're planning to renovate?
hehehe


d:\DEV\WebdevFolder\RealEstateAI\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm




> Entering new AgentExecutor chain...
Action: 
```
{
  "action": "tavily_search_results_json",
  "action_input": {
    "query": "average labor costs for renovation in Herndon, VA"
  }
}
```[{'url': 'https://www.angi.com/companylist/us/va/herndon/kitchen-and-bath-remodeling.htm', 'content': 'The cost of home remodeling is $50,000 for the average homeowner. However, this cost can range anywhere from $3,000 for a small project to $170,000 for a whole home remodel. Factors like project size, location, and room type all affect your overall cost. For example, bedroom remodeling costs are one of the least expensive projects, costing ...'}, {'url': 'https://commonwealthhomedesign.com/how-much-does-a-northern-virginia-home-remodeling-project-cost/', 'content': 'For more information and to start planning your home remodeling project, contact us today: 703-255-9861 or info@commonwealthhome.com. Estimating a Northern Virginia home remodeling project is rarely a straightforward process and requir

In [5]:
from pprint import pprint
print(cc_out)

RENNOVATION QUOTE DOCUMENT

Project Address: 3210 Greenstone Way, Herndon, VA 20171

**Cost Breakdown:**

1. **Labor Costs:**
    - Total Labor Cost: $81,053

2. **Material Costs:**
    - Total Material Cost: $16,150
    - Itemized Costs:
        + Drywall sheets (4'x12'): $200
        + Joint compound: $100
        + Paint (matte finish, soft gray): $300
        + Paintbrushes (various sizes): $100
        + Rollers and extension poles: $100
        + Engineered hardwood flooring (oak, 5"): $3,000
        + Underlayment material (cement-based): $2,400
        + Wood flooring adhesive: $500
        + Flooring nails: $100
        + Cabinets (custom, soft-close): $3,600
        + Granite countertops (3 cm thick): $1,200
        + Backsplash tile (subway, white): $400
        + Kitchen sink (stainless steel, undermount): $300
        + Faucet (touchless, chrome): $150
        + Ceramic tile (white, 12" x 24"): $500
        + Bathroom vanity (48" wide, white): $1,000
        + Granite coun

In [6]:
import os
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx2pdf import convert
from langchain.tools import BaseTool
import re

llm.groq_api_key = random.choice(api_keys)

def add_section(doc, text, font_name='Arial', font_size=12, bold=False, italic=False, alignment='left', color=None):
    # Check if the text should be a bullet point
    if text.startswith("- ") or text.startswith("+ "):
        paragraph = doc.add_paragraph(style='ListBullet')
        text = text[2:]  # Remove the bullet point indicator
    else:
        paragraph = doc.add_paragraph()
    
    run = paragraph.add_run(text)
    run.font.name = font_name
    run.font.size = Pt(font_size)
    run.bold = bold
    run.italic = italic
    if color:
        run.font.color.rgb = RGBColor(*color)
    
    if alignment == 'center':
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    elif alignment == 'right':
        paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    else:
        paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT


def write_file(title, sections, append=False):
    documents_path = os.path.join(os.getcwd(), 'documents')
    os.makedirs(documents_path, exist_ok=True)
    docx_path = os.path.join(documents_path, f'{title}.docx')
    pdf_path = os.path.join(documents_path, f'{title}.pdf')
    
    if append and os.path.exists(docx_path):
        doc = Document(docx_path)
    else:
        doc = Document()

    for section in sections:
        add_section(
            doc,
            section["content"],
            font_name=section.get("font_name", "Arial"),
            font_size=section.get("font_size", 12),
            bold=section.get("bold", False),
            italic=section.get("italic", False),
            alignment=section.get("alignment", "left"),
            color=section.get("color", None)
        )
    
    doc.save(docx_path)
    convert(docx_path, pdf_path)

    return f"Document written and converted to pdf. No further action needed."


class WriteFileTool(BaseTool):
    name = "write_file"
    description = "Writes content into a .docx file with specified formatting and converts it to a .pdf file.\
        If you encounter errors, use this tool ONLY and input everything properly and try again.\
            IMPORTANT! - DO NOT CALL TOOL MULTIPLE TIMES IF FIRST TIME WORKED ALREADY!"
    
    def _run(self, title, sections, append=False):
        self.process_sections_in_batches(title, sections, append)

    def process_sections_in_batches(self, title, sections, append, batch_size=1):
        for i in range(0, len(sections), batch_size):
            batch = sections[i:i+batch_size]
            result = write_file(title, batch, append=(i > 0))
        return ("STOP RIGHT THERE! THAT IS ENOUGH! NO MORE ! YOU ARE DONE!")



def parse_formatted_response(response):
    title_match = re.search(r'"title":\s*"([^"]+)"', response)
    title = title_match.group(1) if title_match else "Untitled Document"
    
    sections_matches = re.findall(r'\{([^}]+)\}', response)
    sections = []
    for match in sections_matches:
        section = {}
        content_match = re.search(r'"content":\s*"([^"]+)"', match)
        font_name_match = re.search(r'"font_name":\s*"([^"]+)"', match)
        font_size_match = re.search(r'"font_size":\s*(\d+)', match)
        bold_match = re.search(r'"bold":\s*(true|false)', match)
        italic_match = re.search(r'"italic":\s*(true|false)', match)
        alignment_match = re.search(r'"alignment":\s*"([^"]+)"', match)
        color_match = re.search(r'"color":\s*\[(\d+),\s*(\d+),\s*(\d+)\]', match)
        
        if content_match:
            content = content_match.group(1)
            # Replace double quotes indicating inches with the word "inch"
            content = re.sub(r'(?<=\d)"', ' inch', content)
            section["content"] = content
        else:
            section["content"] = ""
        
        section["font_name"] = font_name_match.group(1) if font_name_match else "Arial"
        section["font_size"] = int(font_size_match.group(1)) if font_size_match else 12
        section["bold"] = bold_match.group(1) == "true" if bold_match else False
        section["italic"] = italic_match.group(1) == "true" if italic_match else False
        section["alignment"] = alignment_match.group(1) if alignment_match else "left"
        if color_match:
            section["color"] = (int(color_match.group(1)), int(color_match.group(2)), int(color_match.group(3)))
        
        sections.append(section)
    
    append_match = re.search(r'"append":\s*(true|false)', response)
    append = append_match.group(1) == "true" if append_match else False
    
    return title, sections, append



chat_completion = client.chat.completions.create(
    messages=[
        {
            "role":"system",
            "content":"You are a professional formatter. BE ELABORATE. REPLACE all quotations representing inches to the word 'inch'"
        },
        {
            "role":"user",
            "content":"please synthesize ALL of this stuff into sentence format. Start the paragraph with '@' character. put the itemized costs in parentheses beside the item: " + cc_out,
        }
    ],
    model = "llama3-70b-8192",
)

cc_out2 = (chat_completion.choices[0].message.content)
idx = cc_out2.find('@')
cc_out2 = cc_out2[idx+1:].strip()
print('ccount2 old\n',cc_out2)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": """You are a detailed and professional business assistant. Your task is to generate comprehensive and detailed formatted text. 
            Ensure all items, costs, and specifications are included without summarizing or truncating any content.
            Your response must be well-formed and include all details EVEN THE EXPLICIT ADDRESSS!!. Do NOTTTTTTT use phrases like "and so on for the other materials." List every item explicitly.
            You must always output in a structured format as follows:
            
                        You must always output in a structured format as follows:
                        {
                            "title": "The Great Renovation",
                            "sections": [
                                {
                                    "content": "<specific text content>",
                                    "font_name": "<font family>",
                                    "font_size": <integer specifying font size>,
                                    "bold": <true or false>,
                                    "italic": <true or false>,
                                    "alignment": "<right, left, justify or center>"
                                    "color": [<integer for red>, <integer for green>, <integer for blue>]
                                },
                                {
                                    "content": "<another specific text content>",
                                    "font_name": "<another font family>",
                                    "font_size": <another integer specifying font size>,
                                    "bold": <true or false>,
                                    "italic": <true or false>,
                                    "alignment": "<right, left, justify or center>"
                                    "color": [<integer for red>, <integer for green>, <integer for blue>]
                                }
                                // ... more sections as needed
                            ],
                            "append": <true or false> (this indicates appending to document or not)
                        }
                        Ensure the output is well-formed and valid."""
        },
        {
            "role": "user",
            "content": "TITLE OF DOCUMENT IS 'The Great Renovation'. Include titles, headings, format in a nice readable and coherent way, NOT a big block of text.  \
                Specify the formatting for this text. Please make it a \
                    PROFESSIONALLY formatted text WITH black blue colors and times new roman font PLEASE.: " + cc_out2
        }
    ],
    model="llama3-70b-8192",
)

cc_out2 = chat_completion.choices[0].message.content

print('CCOUT2\n',cc_out2)

# title, sections, append = parse_formatted_response(cc_out2)

# write_file_tool._run(title, sections, append)


ccount2 old
 Here is the renovation quote document for the project located at 3210 Greenstone Way, Herndon, VA 20171, which outlines the cost breakdown for the renovation project. The total labor cost is $81,053, and the total material cost is $16,150, which includes (drywall sheets (4'x12'): $200), (joint compound: $100), (paint (matte finish, soft gray): $300), (paintbrushes (various sizes): $100), (rollers and extension poles: $100), (engineered hardwood flooring (oak, 5 inch): $3,000), (underlayment material (cement-based): $2,400), (wood flooring adhesive: $500), (flooring nails: $100), (cabinets (custom, soft-close): $3,600), (granite countertops (3 cm thick): $1,200), (backsplash tile (subway, white): $400), (kitchen sink (stainless steel, undermount): $300), (faucet (touchless, chrome): $150), (ceramic tile (white, 12 inch x 24 inch): $500), (bathroom vanity (48 inch wide, white): $1,000), (granite countertops (3 cm thick): $800), (shower tile (glass, 6 inch x 6 inch): $200), (

In [7]:
from langchain.tools import BaseTool
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import googleapiclient
import google.auth.exceptions

class GoogleDriveUploadTool(BaseTool):
    name = "GoogleDriveUploadTool"
    description = ("Uploads a PDF to Google Drive and sets permissions for a specific user. "
                   "Please set the 'rename' parameter to None if the user does not want to rename the file before uploading "
                   "to Google Drive.")

    def _run(self, file_path: str, user_email: str, rename: str) -> str:
        credentials_path = "D:\\DEV\\WebdevFolder\\realestateai-doc-mgr-051849e19181.json"

        # Authenticate and create the service
        try:
            credentials = service_account.Credentials.from_service_account_file(credentials_path)
            service = build('drive', 'v3', credentials=credentials)
            print("Successfully authenticated and created the service.")
        except google.auth.exceptions.GoogleAuthError as e:
            return f"Authentication error: {e}"

        # File details
        if rename is not None:
            file_metadata = {'name': rename}
        else:
            file_metadata = {'name': os.path.basename(file_path)}

        if not os.path.exists(file_path):
            return f"File not found: {file_path}"
        else:
            media = MediaFileUpload(file_path, mimetype='application/pdf')
            try:
                # Upload file
                file = service.files().create(body=file_metadata, media_body=media, fields='id, name, parents').execute()
                file_id = file.get('id')
                with open('file_id_history.txt','w') as id_hist_file:
                    id_hist_file.write(file_id)
                print(f"File ID: {file_id}")

                # Verify the upload by fetching the file details
                file_info = service.files().get(fileId=file_id, fields='id, name, parents').execute()
                print("File details:")
                print(file_info)

                # Sharing settings
                permission = {
                    'type': 'user',
                    'role': 'writer',  # Set to 'reader' if read-only access is needed
                    'emailAddress': user_email,
                    
                }

                # Grant permission to the specific user
                try:
                    service.permissions().create(
                        fileId=file_id,
                        body=permission,
                        fields='id',
                        sendNotificationEmail=False
                    ).execute()
                    
                    print(f"Granted {permission['role']} access to {user_email}")
                    
                    

                except googleapiclient.errors.HttpError as e:
                    print(f"Error setting permission: {e}")
                    return f"Error setting permission: {e}"

                return f"File uploaded and permissions set successfully. File ID: {file_id}"

            except Exception as e:
                return f"Error during file upload or conversion: {e}"

    

    def _arun(self, file_path: str, user_email: str, rename: str):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")

# Example usage
# tool = GoogleDriveUploadTool()
# response = tool._run(file_path='./documents/The Great Renovation.pdf', user_email='specific-user@example.com', rename=None)
# print(response)

import os
import google.auth
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build


class GoogleSheetsUpdateTool(BaseTool):
    name = "GoogleSheetsUpdateTool"
    description = ("Appends three columns - name, phone number, and link to a PDF - to a preexisting logging google sheet.")
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    credentials_path = 'D:\\DEV\\WebdevFolder\\realestateai-doc-mgr-051849e19181.json'
    spreadsheet_id = '1TSFyiTwctC1tABr2RQouBzLRMCG4RZ7lXdTVi-I58Mo'
    range_name = 'Sheet1'
    
    #creds = authenticate_google_sheets()
    
    def authenticate_google_sheets(self):
        creds = service_account.Credentials.from_service_account_file(
            self.credentials_path, scopes=self.SCOPES)
        return creds
    
    def read_file_content(self, file_path):
        with open(file_path, 'r') as file:
            content = file.read().strip()
        return content
    
    # def extract_file_id(self,output):
    #     match = re.search(r"File ID: ([^'}]+)", output)
    #     if match:
    #         file_id = match.group(1).strip()
    #         return file_id
    #     else:
    #         raise ValueError("File ID not found in the inputted string")
    
    def append_row_to_google_sheets(self, values):
        """
        Appends a row to the specified range in the Google Sheets spreadsheet.
        
        :param values: The data to append, as a list of lists (e.g., [['A1', 'B1', 'C1']]).
        """
        service = build('sheets', 'v4', credentials=self.authenticate_google_sheets())
        sheet = service.spreadsheets()
        body = {'values': values}
        result = sheet.values().append(
            spreadsheetId=self.spreadsheet_id, range=self.range_name,
            valueInputOption="RAW", body=body,
            insertDataOption="INSERT_ROWS"
        ).execute()
        print(f"{result.get('updates').get('updatedCells')} cells appended.")

    def _run(self, name: str, phone_number: str):
        """
        Run the tool to append the row with the given name, phone number, and link to the PDF.
        
        :param name: The full legal name to append.
        :param phone_number: The phone number to append.
        """
        fileID = self.read_file_content('file_id_history.txt')
        
        print(f"Extracted file ID: {fileID}")
        link = f"https://drive.google.com/file/d/{fileID}/view"
        values = [[name, phone_number, link]]
        self.append_row_to_google_sheets(values)
        return "Row appended successfully. PLEASE END THE CHAIN YOU ARE DONE FINISHED DONE!"

    def _arun(self, name: str, phone_number: str):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")

import os
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from langchain.tools import BaseTool
from pydantic import BaseModel, Field, Extra

# If modifying these SCOPES, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

class GmailSendPdfTool(BaseTool):
    name = "GmailSendPdfTool"
    description = "Sends an email with a PDF attachment using Gmail API."
    
    credentials_path: str = Field(..., description="Path to the credentials JSON file")
    
    sender_email: str = Field(..., description="Sender email address")
    recipient_email: str = Field(..., description="Recipient email address")
    subject: str = Field(..., description="Email subject")
    body: str = Field(..., description="Email body")
    pdf_path: str = Field(..., description="Path to the PDF file")
    
    class Config:
        extra = Extra.allow

    def __init__(self, **data):
        super().__init__(**data)
        self.credentials_path = data.get('credentials_path')
        
        self.sender_email = data.get('sender_email')
        self.recipient_email = data.get('recipient_email')
        self.subject = data.get('subject')
        self.body = data.get('body')
        self.pdf_path = data.get('pdf_path')
        self.creds = None
        self.service = None
        self.authenticate()

    def authenticate(self):
        
        if not self.creds or not self.creds.valid:
            if self.creds and self.creds.expired and self.creds.refresh_token:
                self.creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.credentials_path, SCOPES)
                self.creds = flow.run_local_server(port=0)
            
        self.service = build('gmail', 'v1', credentials=self.creds)

    def send_email(self):
        message = MIMEMultipart()
        message['to'] = self.recipient_email
        message['from'] = self.sender_email
        message['subject'] = self.subject
        body_part = MIMEText(self.body, 'plain')
        message.attach(body_part)
        
        part = MIMEBase('application', 'octet-stream')
        with open(self.pdf_path, 'rb') as file:
            part.set_payload(file.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename="{os.path.basename(self.pdf_path)}"')
        message.attach(part)
       
        raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
        raw_message = {'raw': raw_message}

        try:
            message = (self.service.users().messages().send(userId="me", body=raw_message).execute())
            print(f'Message Id: {message["id"]}')
            return message
        except HttpError as error:
            print(f'An error occurred: {error}')
            return None

    def _run(self):
        """
        Run the tool to send the email with the PDF attachment.
        """
        result = self.send_email()
        return "Email sent successfully." if result else "Failed to send email."

    def _arun(self):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")


C:\Users\PC-User\AppData\Local\Temp\ipykernel_7004\166285558.py:191: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  extra = Extra.allow


In [14]:
zapier = ZapierNLAWrapper(zapier_nla_api_key=decipher_suite.decrypt(encrypted_api_key).decode())
toolkit = ZapierToolkit.from_zapier_nla_wrapper(zapier)
import json
from langchain.agents import Tool
my_tools = []
for tool in toolkit.tools:
    my_tools.append(tool)


my_tools.append(
    
    WriteFileTool()
    
)

my_tools.append(
    
    GoogleDriveUploadTool()
    
)

my_tools.append(
    
    GoogleSheetsUpdateTool()
    
)
credentials_path = "D:\\DEV\\WebdevFolder\\client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json"
    
sender_email = 'sriramnallani35@gmail.com'
recipient_email = 'thegreatchrisjohn@gmail.com'
subject = 'RealEstateAI - PDF of Your Renovation Quote'
body = 'Please find the attached PDF document.'
pdf_path = './documents/The Great Renovation.pdf'
my_tools.append(
    
    

    GmailSendPdfTool(
        credentials_path=credentials_path,
        
        sender_email=sender_email,
        recipient_email=recipient_email,
        subject=subject,
        body=body,
        pdf_path=pdf_path
    )
    
)


llm.groq_api_key = random.choice(api_keys)
agent = initialize_agent(my_tools,llm,handle_parsing_errors=True,agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58490%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=Pui4ai0Ng6iejdJMES6UR1b5zisYOx&access_type=offline
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58500%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=vb4lXmilCmvP3BfhSI4c3WsB3A0ASD&access_type=offline
Message Id: 18fb20635ca526a8


In [12]:
title, sections, append = parse_formatted_response(cc_out2)
llm.groq_api_key = random.choice(api_keys)
llm.temperature = 0.2
with ctxmgr.temporary_temperature(llm=llm,new_temp=0.2):
    for i in range(5):
        try:
            result = agent.invoke({"input": f"use ONLY write file tool. INPUT EVERYTHING PROPERLY! --> TITLE: {title}, SECTIONS: {json.dumps(sections)}, APPEND: {append}"})
            if(i >= 5):
                break
            break
        except:
            i += 1
            continue
        

print(result)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "write_file",
  "action_input": {
    "title": "The Great Renovation",
    "sections": [{"content": "Renovation Quote for 3210 Greenstone Way, Herndon, VA 20171", "font_name": "Times New Roman", "font_size": 18, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 0]}, {"content": "Labor Costs", "font_name": "Times New Roman", "font_size": 14, "bold": true, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "$81,053", "font_name": "Times New Roman", "font_size": 14, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Material Costs", "font_name": "Times New Roman", "font_size": 14, "bold": true, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Drywall Sheets (4'x12'): $200", "font_name": "Times New Roman", "font_size": 12, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Joint Compound: $10

100%|██████████| 1/1 [00:00<00:00,  1.00it/s]



Observation: None
Thought:Action:
```
{
  "action": "write_file",
  "action_input": {
    "title": "The Great Renovation",
    "sections": [{"content": "Renovation Quote for 3210 Greenstone Way, Herndon, VA 20171", "font_name": "Times New Roman", "font_size": 18, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 0]}, {"content": "Labor Costs", "font_name": "Times New Roman", "font_size": 14, "bold": true, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "$81,053", "font_name": "Times New Roman", "font_size": 14, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Material Costs", "font_name": "Times New Roman", "font_size": 14, "bold": true, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Drywall Sheets (4'x12'): $200", "font_name": "Times New Roman", "font_size": 12, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Joint Compound: $100", "font_nam

100%|██████████| 1/1 [00:00<00:00,  1.73it/s]



Observation: None
Thought:Action:
```
{
  "action": "write_file",
  "action_input": {
    "title": "The Great Renovation",
    "sections": [{"content": "Renovation Quote for 3210 Greenstone Way, Herndon, VA 20171", "font_name": "Times New Roman", "font_size": 18, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 0]}, {"content": "Labor Costs", "font_name": "Times New Roman", "font_size": 14, "bold": true, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "$81,053", "font_name": "Times New Roman", "font_size": 14, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Material Costs", "font_name": "Times New Roman", "font_size": 14, "bold": true, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Drywall Sheets (4'x12'): $200", "font_name": "Times New Roman", "font_size": 12, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Joint Compound: $100", "font_nam

...


Observation: None
Thought:Action:
```
{
  "action": "write_file",
  "action_input": {
    "title": "The Great Renovation",
    "sections": [{"content": "Renovation Quote for 3210 Greenstone Way, Herndon, VA 20171", "font_name": "Times New Roman", "font_size": 18, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 0]}, {"content": "Labor Costs", "font_name": "Times New Roman", "font_size": 14, "bold": true, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "$81,053", "font_name": "Times New Roman", "font_size": 14, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Material Costs", "font_name": "Times New Roman", "font_size": 14, "bold": true, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Drywall Sheets (4'x12'): $200", "font_name": "Times New Roman", "font_size": 12, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Joint Compound: $100", "font_nam

100%|██████████| 1/1 [00:01<00:00,  1.10s/it]



Observation: None
Thought:Action:
```
{
  "action": "write_file",
  "action_input": {
    "title": "The Great Renovation",
    "sections": [{"content": "Renovation Quote for 3210 Greenstone Way, Herndon, VA 20171", "font_name": "Times New Roman", "font_size": 18, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 0]}, {"content": "Labor Costs", "font_name": "Times New Roman", "font_size": 14, "bold": true, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "$81,053", "font_name": "Times New Roman", "font_size": 14, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Material Costs", "font_name": "Times New Roman", "font_size": 14, "bold": true, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Drywall Sheets (4'x12'): $200", "font_name": "Times New Roman", "font_size": 12, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Joint Compound: $100", "font_nam

100%|██████████| 1/1 [00:00<00:00,  1.12it/s]



Observation: None
Thought:Action:
```
{
  "action": "write_file",
  "action_input": {
    "title": "The Great Renovation",
    "sections": [{"content": "Renovation Quote for 3210 Greenstone Way, Herndon, VA 20171", "font_name": "Times New Roman", "font_size": 18, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 0]}, {"content": "Labor Costs", "font_name": "Times New Roman", "font_size": 14, "bold": true, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "$81,053", "font_name": "Times New Roman", "font_size": 14, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Material Costs", "font_name": "Times New Roman", "font_size": 14, "bold": true, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Drywall Sheets (4'x12'): $200", "font_name": "Times New Roman", "font_size": 12, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Joint Compound: $100", "font_nam

In [13]:
file_path, user_email = "./documents/The Great Renovation.pdf", "thegreatchrisjohn@gmail.com"

import random
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials


file_path = "./documents/The Great Renovation.pdf"
user_email = "sriramnallani35@gmail.com"


llm.groq_api_key = random.choice(api_keys)
with ctxmgr.temporary_temperature(llm,0.2):
    result = agent.invoke({"input": f"please use google drive upload tool, rename the file to 'Quote' --> FILE_PATH: {file_path} USER_EMAIL: {user_email}"})
    print(result)

    name = 'Johng Doe23'
    phone_number = '123455444455467890'
    result = agent.invoke({"input": f"please use google sheets update tool, input these into the tool PROPERLY! --> {name}, {phone_number}"})
    print(result)

    result = agent.invoke({"input": f"please use gmail pdf send tool"})
    print(result)





> Entering new AgentExecutor chain...
Action:
```
{
  "action": "GoogleDriveUploadTool",
  "action_input": {
    "file_path": "./documents/The Great Renovation.pdf",
    "user_email": "sriramnallani35@gmail.com",
    "rename": "Quote"
  }
}
```
Successfully authenticated and created the service.
File ID: 1IAAVoKHJNS9QxoGEg-KInpnVTBY6XC7G
File details:
{'id': '1IAAVoKHJNS9QxoGEg-KInpnVTBY6XC7G', 'name': 'Quote', 'parents': ['0AJW9tsOpMsePUk9PVA']}
Granted writer access to sriramnallani35@gmail.com

Observation: File uploaded and permissions set successfully. File ID: 1IAAVoKHJNS9QxoGEg-KInpnVTBY6XC7G
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "File uploaded and permissions set successfully. File ID: 1IAAVoKHJNS9QxoGEg-KInpnVTBY6XC7G"
}
```


> Finished chain.
{'input': "please use google drive upload tool, rename the file to 'Quote' --> FILE_PATH: ./documents/The Great Renovation.pdf USER_EMAIL: sriramnallani35@gmail.com", 'output': 'File uploaded and permissi

TypeError: _run() got an unexpected keyword argument 'recipient_email'